Importiamo tutte le librerie che ci interessano

In [2]:
import gym
import random
import numpy as np
from keras.layers import Input, Conv2D, Dense, MaxPooling2D , Flatten
from keras.models import Model, Sequential, load_model
from collections import deque
from keras.optimizers import Adam
import keras
import matplotlib.pyplot as plt

Using TensorFlow backend.


Creo l'ambiente SpaceInvaders-vo

In [3]:
env = gym.make('SpaceInvaders-v0')

Creiamo la nostra rete neurale

In [5]:
class DQN:
    def __init__(self, input_shape, output_shape, discount=0.99, update_target_every=10, memory_size=2000):
        self.input_shape=input_shape
        self.output_shape=output_shape
        self.discount=discount
        self.update_target_every=update_target_every
        self.policy_net=self.create_model()
        self.memory=deque(maxlen=memory_size)
        self.target_counter=0 
    
    def create_model(self):
        model=Sequential()
        model.add(Conv2D(input_shape=self.input_shape, filters=16, kernel_size=(8,8), strides=(4,4), padding="valid", 
                        activation="relu", use_bias=True,))
        model.add(Conv2D(filters=16, kernel_size=(4,4), strides=(2,2), padding="valid", 
                       activation="relu", use_bias=True,))
        #model.add(Conv2D(filters=32, kernel_size=(2,2), padding="valid", 
        #                activation="relu", use_bias=True,))
        model.add(Flatten())
        model.add(Dense(128, activation="relu"))
        model.add(Dense(self.output_shape)) # era softmax
        adm=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
        model.compile(loss="mse", optimizer=adm, metrics=["accuracy"] )
        return model        

Definiamo tre metodi per preprocessare la nostra immagine croppandola e portandola ad una scala di grigi

In [6]:
def to_greyscale(img):
        return np.mean(img , axis=2).astype(np.uint8)

In [7]:
def downsample(img):
        return img[::2 , ::2]

In [8]:
def crop(img):
    return img[10:100 ,:]

In [9]:
img = env.reset()
plt.imshow(preprocess(img))
#print(preprocess(img).shape)

NameError: name 'preprocess' is not defined

In [ ]:
plt.imshow(to_greyscale(downsample(img)))

In [20]:
def preprocess(img):
        return crop(to_greyscale(downsample(img)))/255

Definiamo un metodo per trasformare ogni ricompensa in (-1, 0, +1)

In [ ]:
def transform_reward(reward):
    return np.sign(reward)

In [13]:
class Memory:
    def __init__(self , memory = [] , maxsize = 10000):
            self.iteration = 0
            self.memory = memory
            self.maxsize = maxsize

In [14]:
def iteration(env  , model  , mem):
    env.reset()
    epsilon =  (0.995)**(mem.iteration)
    o,r,d,i = env.step(0)
    env.render()
    processed_state = preprocess(o)
    processed_state = processed_state.reshape((1,90,80,1))
    mem.memory = []
    for i in range(mem.maxsize):
        if  random.random() < epsilon:
            ac = env.action_space.sample()
        else:
            ac = np.argmax(model.policy_net.predict(processed_state))
        next_state,reward,done,info = env.step(ac)
        if done:
            next_processed_state = preprocess(next_state)
            next_processed_state = next_processed_state.reshape((1,90,80,1))
            mem.memory.append((processed_state,ac  , next_processed_state , reward, done ))
            break
        env.render()
        next_processed_state = preprocess(next_state)
        next_processed_state = next_processed_state.reshape((1,90,80,1))
        mem.memory.append((processed_state,ac  , next_processed_state , reward, done ))
        processed_state=next_processed_state
    mem.iteration=mem.iteration+1   

In [15]:
mem = Memory()
dqn = DQN((90,80,1),6)

In [16]:
dqn.policy_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 21, 19, 16)        1040      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 8, 16)          4112      
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               147584    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 774       
Total params: 153,510
Trainable params: 153,510
Non-trainable params: 0
_________________________________________________________________


In [10]:
iteration(env , dqn , mem)

NameError: name 'iteration' is not defined

In [ ]:
env.close()

In [ ]:
len(mem.memory)

In [ ]:
iteration(env , dqn , mem)

In [ ]:
len(mem.memory)

In [ ]:
mem.iteration

In [ ]:
dqn.policy_net.predict(mem.memory[0][0])

In [ ]:
for index, _  in enumerate(mem.memory):
    print(index , dqn.policy_net.predict(mem.memory[index][0]))

In [ ]:
for index, _  in enumerate(mem.memory):
    print(index ,mem.memory[index][3])

In [ ]:
env.close()

In [17]:
def train ( mem , model,gamma = 0.99 ):
    i=0
    for state , action , next_state , reward ,done in mem.memory:
        i+=1
        if i >44:
            #rewards = np.zeros(6)
            #rewards[action]=reward
            #q_values = np.zeros(6)
            #q_values[best]=1
            #target = rewards+ (  gamma * q_values)
            #target = target.reshape(1,6)
            target =model.policy_net.predict(state)[0]
            if not done:
                #best = np.argmax(model.policy_net.predict(next_state))
                target[action]= reward + gamma* np.max(model.policy_net.predict(next_state)[0])
            else:
                target[action]= 0
            target =target.reshape(1,6)
            model.policy_net.fit(state , target , verbose=0)

In [18]:
train(mem , dqn)

In [21]:
iteration(env,dqn,mem)
env.close()

In [22]:
len(mem.memory)

353

In [23]:
mem = Memory()
dqn = DQN((90,80,1),6)

In [ ]:
for i in range(10000):
    train(mem , dqn)
    iteration(env,dqn,mem)
    env.close()
    print(f"iterazione: {mem.iteration}\t memoria: { len(mem.memory)}\t esplorazione: {(0.995)**(mem.iteration+1)}")
    #print("memoria" , len(mem.memory))
    #print("esplorazione" ,(0.995)**(mem.iteration+1))

iterazione: 1	 memoria: 419	 esplorazione: 0.990025








iterazione: 2	 memoria: 602	 esplorazione: 0.985074875
iterazione: 3	 memoria: 691	 esplorazione: 0.980149500625
iterazione: 4	 memoria: 1002	 esplorazione: 0.975248753121875
iterazione: 5	 memoria: 743	 esplorazione: 0.9703725093562656
iterazione: 6	 memoria: 816	 esplorazione: 0.9655206468094842
iterazione: 7	 memoria: 402	 esplorazione: 0.9606930435754368
iterazione: 8	 memoria: 623	 esplorazione: 0.9558895783575596
iterazione: 9	 memoria: 1269	 esplorazione: 0.9511101304657719
iterazione: 10	 memoria: 849	 esplorazione: 0.946354579813443
iterazione: 11	 memoria: 669	 esplorazione: 0.9416228069143757
iterazione: 12	 memoria: 669	 esplorazione: 0.9369146928798039
iterazione: 13	 memoria: 807	 esplorazione: 0.9322301194154049
iterazione: 14	 memoria: 421	 esplorazione: 0.9275689688183278
iterazione: 15	 memoria: 410	 esplorazione: 0.9229311239742362
iterazione: 16	 memoria: 931	 esplorazione: 0.918316468354365
iterazione: 17	

iterazione: 108	 memoria: 551	 esplorazione: 0.5790496471185969
iterazione: 109	 memoria: 672	 esplorazione: 0.5761543988830038
iterazione: 110	 memoria: 1218	 esplorazione: 0.5732736268885888
iterazione: 111	 memoria: 359	 esplorazione: 0.5704072587541459
iterazione: 112	 memoria: 827	 esplorazione: 0.5675552224603752
iterazione: 113	 memoria: 935	 esplorazione: 0.5647174463480732
iterazione: 114	 memoria: 846	 esplorazione: 0.5618938591163328
iterazione: 115	 memoria: 395	 esplorazione: 0.5590843898207511
iterazione: 116	 memoria: 448	 esplorazione: 0.5562889678716475
iterazione: 117	 memoria: 645	 esplorazione: 0.5535075230322892
iterazione: 118	 memoria: 696	 esplorazione: 0.5507399854171278
iterazione: 119	 memoria: 829	 esplorazione: 0.5479862854900421
iterazione: 120	 memoria: 793	 esplorazione: 0.5452463540625919
iterazione: 121	 memoria: 612	 esplorazione: 0.542520122292279
iterazione: 122	 memoria: 796	 esplorazione: 0.5398075216808176
iterazione: 123	 memoria: 1632	 esploraz

In [ ]:
def play_network(env , model):
    env.reset()
    o,r,d,i = env.step(0)
    env.render()
    processed_state = preprocess(o)
    processed_state = processed_state.reshape((1,90,80,1))
    for i in range(10000):
        ac = np.argmax(model.policy_net.predict(processed_state))
        next_state,reward,done,info = env.step(ac)
        if done:
            break
        env.render() 

In [ ]:
play_network(env , dqn)

In [ ]:
env.close()

In [ ]:
#salviamo la rete
dqn.policy_net.save("modello_prova.h5")

In [ ]:
plt.imshow(mem.memory[540][0].reshape((90,80)))